# Installation and Basic Operations Notes for Kubernetes (the OS of the cloud)

- https://www.docker.com/products/docker-desktop
- https://minikube.sigs.k8s.io/docs/tutorials/

## Introduction to Kubernetes
**Kubernetes (K8s)** (K8s -> "kates") is an orchestrator of cloud-native microservices applications. The name **Kubernetes** comes from the Greek word meaning `helmsman`, a nautical/sailing term for the person that steers a ship. The wheel of a ship is also called the `helm`. **Microservices** application is made of lots of small, specialised parts that are losely coupled to create a usefyl application. **Cloud-native** apps can self-heal, automatically scale, and be updated without downtime. An **orchestrator** brings together a set of microservices and organises them into an application that brings value, provides and manages cloud-native features such as scaling, self-healing, and updates. 

**Kubernetes** does an amazing job of abstracting underlying cloud and server infrastructure, so you don't have to care what cloud or servers your apps are running on. **Kuberenetes** is called `the operating system of the cloud`. Applications written for Kubernetes will run anywhere that you have Kubernetes. K8s apps can:
- be deployed to one cloud today and be re-deployed to another tomorrow,
- run multi-cloud,
- be more easily ramp onto a cloud and then ramp off back to on-prem.

We call a Kubernetes installation (one or more machines installed with Kubernetes) a `Kubernetes cluster`. The machines can be physical servers, virtual machines, cloud instances, a laptop, Raspberry Pi, and more. We refer to machines in K8s cluster as `nodes`. There are two types of nodes: master and worker. `Masters` (master nodes) host the `control plane` (brains of the cluster) and (worker) `Nodes` are where you run user applications.

**Masters** run many services that form the control pane but the `API Server` is the only part of Kubernetes cluster you directly interact with. **Nodes** run Linux or Windows user applications. The `Scheduler` chooses which Nodes to run user applications on. The `Store` is where the state of the cluster and all applications is stored. The `Cloud controller` allows K8s to integrate with cloud services such as storage and load-balancers.

All Nodes run a couple of services including `Kubelet` (main Kubernetes agent) and `Container runtime` (starts and stops containers). Most of the day-to-day management of K8s cluster is done using the Kubernetes command line toll called **kubectl** ("kube see tee ell").

**Kubernetes** came into existence in the summer of 2014 as an open-source next gext generation of Borg and Omega (Google's internal tools) for managing containers.

## Kubernetes as a Managed Service

- [Linode](https://www.linode.com/)
- [AWS Elastic Kubernetes Service (EKS)]()
- [Azure Kubernetes Service (AKS)]()
- [Google Kubernetes Engine (GKE)]()

## Getting Kubernetes

### Integrated with Docker Desktop
With [Docker Desktop](https://www.docker.com/products/docker-desktop) you get Docker runtime and a single-node Kubernetes cluster that is great for development and learning, and kubectl command line utility. On Mac OS your single-node K8s cluster will run inside a lightweight VM, on WIndows it can run natively on WSL2 (Windows Subsystem for Linux, ver. 2). Enable Kubernetes in Docker Desktop preferences.
```
# Verify Docker and K8s installation
docker --version
kubectl version -o yaml
```

### Minikube
[minikube](https://minikube.sigs.k8s.io/docs/start/) quickly sets up a local Kubernetes cluster on macOS, Linux, and Windows.
```
# Choose the rith version from: https://minikube.sigs.k8s.io/docs/start/
#curl -LO https://storage.googleapis.com/minikube/releases/v1.16.0/minikube-linux-amd64
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

# Verify Docker and K8s installation
docker --version
kubectl version -o yaml

# Test
minikube status
minikube start
kubectl get po -A

minikube dashboard

kubectl create deployment hello-minikube --image=k8s.gcr.io/echoserver:1.4
kubectl expose deployment hello-minikube --type=NodePort --port=8080

kubectl get services hello-minikube

# The easiest way to access this service is to let minikube launch a web browser for you
minikube service hello-minikube

# Your application is now available at http://localhost:7080/.
kubectl port-forward service/hello-minikube 7080:8080

minikube stop
minikube delete --all
```

### Kubectl
You may already have kubectl installed via another tool such as Docker Desktop or minikube.
```
# Mac OS
kubectl version --client -o yaml
brew install kubectl

# kubeconfig file is located (in order to connect to a cluster, take kubeconfig file from the provider and copy it here
~/.kube/config

kubectl get nodes
```

## Install Homebrew
https://www.how2shout.com/linux/how-to-install-brew-ubuntu-20-04-lts-linux/
```
# https://medium.com/@edwardbaeg9/using-homebrew-on-windows-10-with-windows-subsystem-for-linux-wsl-c7f1792f88b3
sh -c "$(curl -fsSL https://raw.githubusercontent.com/Linuxbrew/install/master/install.sh)"

echo 'eval "$(/home/linuxbrew/.linuxbrew/bin/brew shellenv)"' >> ~/.profile
eval "$(/home/linuxbrew/.linuxbrew/bin/brew shellenv)"
sudo apt-get install build-essential
brew install gcc
```

.profile## pyenv - a tool for managing multiple Python versions:

1. Install Python in your user space
1. Install multiple versions of Python
1. Specify the exact Python version you want
1. Switch between the installed versions

https://realpython.com/intro-to-pyenv/
```
$ python -V
$ which python

$ python3 -V
$ which python

# Install dependencies
$ brew install openssl readline sqlite3 xz zlib

# pyenv installer
$ brew install pyenv

$ echo export PATH="$HOME/.pyenv/bin:$PATH" >> ~/.profile

# run the command 'pyenv init --path', paste the output in your shell (~/.profile) at the end line, to enable shims.
# and after that add 'eval "$(pyenv init -)" '
# source ~/.profile
# from https://github.com/pyenv/pyenv/issues/849 (biata 122 commented on 17 Jun)
# or just run this:
$ echo 'eval "$(pyenv init --path)"' >> ~/.profile 

$ echo 'eval "$(pyenv init -)"' >> ~/.profile
#$ echo 'eval "$(pyenv virtualenv-init -)"' >> ~/.profile

# Reload your shell
$ exec "$SHELL" # Or just restart your terminal

# Remove pyenv
#$ brew uninstall pyenv

# Install Python (from the source)
$ pyenv install --list | grep " 3\.[89]"
$ pyenv install -v 3.9.6

# Run installation tests
$ python -m test

# Installation location
$ ls ~/.pyenv/versions/

# Remove versions
#$ rm -rf ~/.pyenv/versions/3.9.6
#$ pyenv uninstall 3.9.6

# Available versions
$ pyenv versions

# Actual path to Python
$ pyenv which python

# Select Python version to use
$ pyenv global 3.9.6
#$ pyenv local 2.7.15 # for specific folder/app
#$ pyenv shell 2.7.15 # for specific shell
$ python -V (shows system/global version)
$ pyenv versions

# Show current version
$ pyenv version (shows global pyenv/current version)

# Show current pip version (full path)
$ pyenv which pip
```

## pyenv-virtualenv - Python virtual environments management with pyenv

GitHub - pyenv/pyenv-virtualenv: a pyenv plugin to manage virtualenv (a.k.a. python-virtualenv): https://github.com/pyenv/pyenv-virtualenv

Python Virtual Environments: A Primer – Real Python: https://realpython.com/python-virtual-environments-a-primer/ 

- pyenv manages multiple versions of Python itself.
- virtualenv/venv manages virtual environments for a specific Python version.
- pyenv-virtualenv manages virtual environments for across varying versions of Python.

```
# Should be already installed with pyenv
$ brew install pyenv-virtualenv

# Create a virtual environment 
# pyenv virtualenv <python_version> <environment_name>
$ pyenv virtualenv 3.9.6 myproject

# Activate your environment
# This creates a .python-version file in your current working directory and because you ran eval "$(pyenv virtualenv-init -)" in your environment, the environment will automatically be activated.
$ pyenv local myproject

# Verify a new project-specific version of Python is created
$ pyenv which python
$ pyenv which pip
$ pyenv versions

# If you did not configure eval "$(pyenv virtualenv-init -)" to run in your shell, you can manually activate/deactivate your Python versions.
# The above is what pyenv-virtualenv is doing when it enters or exits a directory with a .python-version file in it.
$ pyenv activate <environment_name>
$ pyenv deactivate
```

## Setup Jupyter Lab Python/Python environment
In the terminal/shell
```
#pip install virtualenv
#pip install virtualenvwrapper
#brew install pyenv-virtualenv

#Create a new virtualenv named tfx with Python 3.7.11
pyenv virtualenv 3.7.11 tfx3.7.11

pyenv versions

pyenv shell tfx3.7.11

pyenv which python
python -V

pip install tfx
pip install apache-beam[interactive]

#Making Jupyter Lab kernel linking to the proper python environment
#https://stackoverflow.com/questions/28831854/how-do-i-add-python3-kernel-to-jupyter-ipython/44072803#44072803
jupyter kernelspec list
jupyter --paths
/home/ksatola/.pyenv/versions/tfx3.7.11/bin/python -m ipykernel install --prefix=/home/ksatola/.local --name 'tfx3.7.11'
#jupyter kernelspec uninstall python3

pyenv shell tfx3.7.11 ; jupyter lab --no-browser
```

refs: 
- https://stackoverflow.com/questions/28831854/how-do-i-add-python3-kernel-to-jupyter-ipython/44072803#44072803
- https://ipython.readthedocs.io/en/6.5.0/install/kernel_install.html#kernels-for-different-environments
- https://www.alfredo.motta.name/create-isolated-jupyter-ipython-kernels-with-pyenv-and-virtualenv/
- https://innodesign.io/pythons-virtual-environments/
- https://janakiev.com/blog/jupyter-virtual-envs/